2. 평가
- 심사 기준 : NMAE-10(Normalized Mean Absolute Error)

- Public 평가 :
학습용 제공 데이터를 이용하여 미래 한 달간 발전량 예측 후 평가
- Private 평가 :
대회 종료 시점부터 30일간 실제 발전량을 하루씩 평가
1일 1회 채점 후 누적 결과 리더보드 반영
Private 평가 기간 제출물 업데이트 가능 (※제출 후 선택 파일 확인을 반드시 해주시기 바랍니다.)

In [ ]:
# 모듈
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# 샘플링
from sklearn.model_selection import train_test_split

# RF
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# lightgbm, svm ...

# LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

# 측정
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [ ]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 1 데이터 불러오기

In [ ]:
# 관측 데이터 (imputed)
all_obs_imputed = pd.read_csv('/content/drive/MyDrive/전주시 데이터분석/울산/ulsan_all_obs_imputed.csv')
all_obs_imputed_minmax = pd.read_csv('/content/drive/MyDrive/전주시 데이터분석/울산/ulsan_all_obs_imputed_minmax.csv')
# 관측 데이터
all_obs = pd.read_csv('/content/drive/MyDrive/전주시 데이터분석/울산/ulsan_all_obs.csv')

# 에너지 데이터
energy = pd.read_csv('/content/drive/MyDrive/전주시 데이터분석/data/energy.csv')

In [ ]:
#발전량 시간 1시 ->  0시로 바꾸기
def convert_time(x):
    date, hr = x.split(' ')
    h,m,s = hr.split(':')
    h = str(int(h)-1)
    hr = ':'.join([h,m,s])
    return ' '.join([date, hr])

energy['time']=energy['time'].apply(convert_time)
energy['time']=pd.to_datetime(energy['time'])

all_obs['energy']=energy[['ulsan']]
# all_obs=pd.concat([all_obs,ulsan], axis=1)

In [ ]:
all_obs['pm10'] = all_obs['pm10'].apply(lambda x: np.nan if x =='-' else x)
all_obs['pm10'] = all_obs['pm10'].astype('float')

In [ ]:
all_obs.drop(['Unnamed: 0'], axis=1, inplace=True)
all_obs['time'] = pd.to_datetime(all_obs['time'])

In [ ]:
all_obs['hour'] = all_obs['time'].dt.hour
all_obs['day'] = all_obs['time'].dt.day

In [ ]:
all_obs.shape

In [ ]:
corr = all_obs.corr(method='pearson')
#corr.style.background_gradient(cmap='coolwarm')
fig, ax = plt.subplots( figsize=(17,8) )
sns.heatmap(corr, cmap='RdYlBu_r', annot = True, linewidths=.5)
plt.show()

In [ ]:
all_obs['energy'].plot()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(all_obs.loc[:120, 'time'], all_obs.loc[:120,'sun_hr'])
plt.plot(all_obs.loc[:120,'time'], all_obs.loc[:120,'solar'])
plt.legend(['sun_hr','solar'])

In [ ]:
all_obs.describe()

In [ ]:
seasonal = all_obs[['time','winddir','humid','cloud',	'sun_hr',	'solar', 'ozone',	'no2',	'pm2.5',	'pm10',	'rn',	'energy']]

In [ ]:
seasonal =seasonal.set_index('time')

In [ ]:
seasonal = seasonal.replace(np.NaN,0)

In [ ]:
seasonal2 = seasonal[['sun_hr',	'solar','rn',	'energy','ozone','humid']]

In [ ]:
res3

In [ ]:
import math

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
res2 = seasonal_decompose(seasonal2.loc['2018-03-01':'2018-03-18'], model='applicative')

In [ ]:
res3 = res2.seasonal

In [ ]:
res3['energy']  = res3['energy'].values * 0.5

In [ ]:
res3.plot(figsize=(20,8))

In [ ]:
fig, axes=plt.subplots(3,1, figsize=(20,10))
res.observed.plot(ax=axes[0])
res.trend.plot(ax=axes[1])
res.seasonal.plot(ax=axes[2])

## 1-1 데이터 정리

In [ ]:
#발전량 시간 1시 ->  0시로 바꾸기
def convert_time(x):
    date, hr = x.split(' ')
    h,m,s = hr.split(':')
    h = str(int(h)-1)
    hr = ':'.join([h,m,s])
    return ' '.join([date, hr])

energy['time']=energy['time'].apply(convert_time)
energy['time']=pd.to_datetime(energy['time'])

all_obs_imputed_minmax['energy']=energy[['ulsan']]
# all_obs=pd.concat([all_obs,ulsan], axis=1)

In [ ]:
all_obs_imputed_minmax

# 에너지 데이터

In [ ]:
plt.figure(figsize=(30,5))
plt.plot(energy.loc[:800,'time'], energy.loc[:800,'ulsan'], '.-')

In [ ]:
import seaborn as sns

In [ ]:
corr = all_obs_imputed_minmax.corr(method='pearson')
fig, ax = plt.subplots( figsize=(15,7) )
sns.heatmap(corr, cmap='RdYlBu_r', annot = True, linewidths=.5)
plt.show()

# 데이터셋 나누기

In [ ]:
all_obs.columns

In [ ]:
x = all_obs_imputed_minmax[['temp',	'windsp','winddir', 'humid','so2','ozone', 'co', 'no2', 'pm10', 'pm25', 'rn', 'month', 'cloud']]
y = all_obs_imputed_minmax[['energy']]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, test_size=0.3, shuffle=False)

In [ ]:
print(x_test.shape,  y_test.shape, x_train.shape, y_train.shape)

In [ ]:
all_obs_imputed_minmax.isnull().sum()

# Random Forest

In [ ]:
 # 학습 진행
forest = RandomForestRegressor(n_estimators=100, max_depth=12, min_samples_leaf= 8, min_samples_split= 8)
model = forest.fit(x_train, y_train)

# 예측
y_pred = forest.predict(x_test)
print(y_pred)
print(list(y_test))
#accuracy = accuracy_score(y2_test, y_pred)

# 정확도 확인
# print(mean_squared_error(y2_test, y_pred))
print(forest.score(x_test, y_test))
#print(accuracy)

In [ ]:
params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestRegressor(random_state =10, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(x_train, y_train)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
print("특성 중요도 : \n{}".format(forest.feature_importances_))
feature_names = x_train.columns.tolist()

def plot_feature_importances(model):

    n_features = x_train.shape[1]
    plt.barh(range(n_features), model.feature_importances_)

    plt.yticks(np.arange(n_features),feature_names)

    plt.xlabel("attr importances")

    plt.ylabel("attr")

plt.show()
plot_feature_importances(forest)

# LSTM

In [ ]:
TEST_SIZE = 25631
WINDOW_SIZE = 672

train = all_obs_imputed_minmax[:-TEST_SIZE]
test = all_obs_imputed_minmax[-TEST_SIZE:]

In [ ]:
def make_dataset(all_obs_imputed_minmax, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(all_obs_imputed_minmax) - window_size):
        feature_list.append(all_obs_imputed_minmax.iloc[i:i+window_size])
        label_list.append(label.iloc[i+window_size])
    return np.array(feature_list), np.array(label_list)

In [ ]:
feature_cols = ['temp',	'windsp','winddir', 'humid','so2','ozone', 'co', 'no2', 'pm10', 'pm25', 'rn', 'month', 'cloud']
label_cols = ['energy']

train_feature = train[feature_cols]
train_label = train[label_cols]

train_feature, train_label = make_dataset(train_feature, train_label, 20)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
x_train.shape, x_valid.shape

In [ ]:
test_feature, test_label = make_dataset(y_test

In [ ]:
model = Sequential()
model.add(LSTM(16, 
               input_shape=(x_train.shape[1], x_train.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )

model.add(Dense(1))

In [ ]:
import os

model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)

model_path = 'model'
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(x_train, y_train, 
                                    epochs=200, 
                                    batch_size=16,
                                    validation_data=(x_valid, y_valid), 
                                    callbacks=[early_stop, checkpoint])

# 평가 (NMAE)

In [ ]:
def sola_nmae(answer_df, submission_df):
    submission = submission_df[submission_df['time'].isin(answer_df['time'])]
    submission.index = range(submission.shape[0])
    
    # 시간대별 총 발전량
    sum_submission = submission.iloc[:,1:].sum(axis=1)
    sum_answer = answer_df.iloc[:,1:].sum(axis=1)
    
    # 발전소 발전용량
    capacity = {
        'dangjin_floating':1000, # 당진수상태양광 발전용량
        'dangjin_warehouse':700, # 당진자재창고태양광 발전용량
        'dangjin':1000, # 당진태양광 발전용량
        'ulsan':500 # 울산태양광 발전용량
    }
    
    # 총 발전용량
    total_capacity = np.sum(list(capacity.values()))
    
    # 총 발전용량 절대오차
    absolute_error = (sum_answer - sum_submission).abs()
    
    # 발전용량으로 정규화
    absolute_error /= total_capacity
    
    # 총 발전용량의 10% 이상 발전한 데이터 인덱스 추출
    target_idx = sum_answer[sum_answer>=total_capacity*0.1].index
    
    # NMAE(%)
    nmae = 100 * absolute_error[target_idx].mean()
    
    return nmae